## 🗺️ Milieuschutz Zones Visualization Notebook

This notebook loads the enriched Milieuschutz zones and Berlin district boundaries and displays them on an interactive map using **Folium**.

### 🎯 What will be shown on the map:

- 🏙️ **Berlin Bezirke (neighborhoods)** as a light grey background layer
- 🛡️ **Milieuschutz zones**:
  - Color-coded by type (`type_en`)
  - Tooltips showing both zone type and neighborhood (`bezirk`)
- 🧭 Interactive layer control

---

### 📦 Required Files:

- `milieuschutz_enriched.geojson` – contains the Milieuschutz zones with `type_en`
- `berlin_neighborhoods_clean.geojson` – contains the Berlin neighborhoods boundaries

💡 This map is useful for analysis, presentations, and understanding urban protection across Berlin.

In [1]:
import folium
import geopandas as gpd
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# 💾 Load data
enriched_df = gpd.read_file("/Users/zeal.v/Help/layered-populate-data-pool-da/milieuschutz/sources/milieuschutz_enriched.geojson")
berlin_neighborhoods = gpd.read_file("/Users/zeal.v/Help/layered-populate-data-pool-da/milieuschutz/sources/berlin_neighborhoods_clean.geojson")

# ✅ Filter only official Bezirke
valid_bezirke = [
    "Mitte", "Friedrichshain-Kreuzberg", "Pankow", "Charlottenburg-Wilmersdorf",
    "Spandau", "Steglitz-Zehlendorf", "Tempelhof-Schöneberg", "Neukölln",
    "Treptow-Köpenick", "Lichtenberg", "Reinickendorf", "Marzahn-Hellersdorf"
]
filtered_bezirke = berlin_neighborhoods[berlin_neighborhoods["neighborhood"].isin(valid_bezirke)]

# 🎨 Set up color mapping from tab10
unique_types = enriched_df["type_en"].unique()
cmap = cm.get_cmap("tab10", len(unique_types))
type_color_map = {t: mcolors.to_hex(cmap(i)) for i, t in enumerate(unique_types)}

# 🗺️ Create folium map
m = folium.Map(location=[52.52, 13.405], zoom_start=10, tiles="OpenStreetMap")

# 🩶 Add Berlin Bezirke
folium.GeoJson(
    data=filtered_bezirke,
    style_function=lambda x: {
        "fillColor": "lightgray",
        "color": "black",
        "weight": 1.0,
        "fillOpacity": 0.5
    },
    tooltip=folium.GeoJsonTooltip(fields=["neighborhood"]),
    name="Berlin Bezirke (Project Scope)"
).add_to(m)

# 🛡️ Add Milieuschutz zones
folium.GeoJson(
    data=enriched_df,
    style_function=lambda f: {
        "fillColor": type_color_map.get(f["properties"]["type_en"], "gray"),
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 1
    },
    tooltip=folium.GeoJsonTooltip(fields=["type_en", "bezirk"]),
    name="Milieuschutz Zones (by Type)"
).add_to(m)

# 🧭 Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# 🖼️ Show map
m

/var/folders/9l/hrrtsvvx7_v1n1nd2s2h8k4r0000gn/T/ipykernel_73480/148795638.py:20: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("tab10", len(unique_types))


In [2]:
# 💾 Save the map to an HTML file
m.save("/Users/zeal.v/Help/layered-populate-data-pool-da/milieuschutz/sources/milieuschutz_map.html")